In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
overrides = pd.read_excel(r"C:\Users\n740789\Documents\Projects_local\DataSets\overrides\202502_BBDD_Overrides.xlsx",
                   sheet_name="MAPEO P-S",
                   dtype={'permId (Clarity)':str, 'ClarityID':str},
                   )

In [3]:
overrides.columns

Index(['#', 'ClarityID', 'permId (Clarity)', 'IssuerID', 'Issuer Name',
       'SNT-WORLD\n(Y/-)', 'PARENT NAME (Aladdin)',
       'PARENT IssuerID (Aladdin)', 'Parent (P) / Subsidiaria (S)',
       'PARENT NAME (Clarity)', 'Parent ClarityID', 'GICS_2', 'GICS_2\nParent',
       'Sustainability Rating Parent', 'OVR Inheritance BiC',
       'OVR STR001\n(SEC)', 'OVR STR002\n(EC*)', 'OVR STR003\n(EC)',
       'OVR STR003B\n(EC*)', 'OVR STR004\n(SEC)', 'OVR STR005\n(EC*)',
       'OVR STR006\n(SEC)', 'OVR STR007\n(SEC)', 'OVR CS001\n(SEC*)',
       'OVR CS002\n(EC*)', 'OVR CS003\n(SEC)', 'OVR ARTICULO 8\n(C)',
       'Motivo principal OVR', 'Motivo secundario OVR',
       'Detalle OVR/Observación adicional', 'Fecha aplicación OVR'],
      dtype='object')

In [5]:
overrides.columns = (
    overrides.columns.str.strip()
    .str.replace(' ', '_')
    .str.replace(")","")\
    .str.replace("(","")
    .str.replace("/","")
    .str.lower()
    .str.replace("\n","_")
    .str.replace("*","")
    .str.replace("__","_")
    )

In [6]:
overrides.columns

Index(['#', 'clarityid', 'permid_clarity', 'issuerid', 'issuer_name',
       'snt-world_y-', 'parent_name_aladdin', 'parent_issuerid_aladdin',
       'parent_p_subsidiaria_s', 'parent_name_clarity', 'parent_clarityid',
       'gics_2', 'gics_2_parent', 'sustainability_rating_parent',
       'ovr_inheritance_bic', 'ovr_str001_sec', 'ovr_str002_ec',
       'ovr_str003_ec', 'ovr_str003b_ec', 'ovr_str004_sec', 'ovr_str005_ec',
       'ovr_str006_sec', 'ovr_str007_sec', 'ovr_cs001_sec', 'ovr_cs002_ec',
       'ovr_cs003_sec', 'ovr_articulo_8_c', 'motivo_principal_ovr',
       'motivo_secundario_ovr', 'detalle_ovrobservación_adicional',
       'fecha_aplicación_ovr'],
      dtype='object')

In [7]:
ovr_filtered = overrides[
    [
        "clarityid", 
        "permid_clarity",
        "issuerid",
        "issuer_name",
        "parent_p_subsidiaria_s",
        "parent_name_clarity",
        "parent_clarityid",
        "parent_name_aladdin",
        "parent_issuerid_aladdin",
        "ovr_str001_sec",
        "ovr_str002_ec",
        "ovr_str003_ec",
        "ovr_str003b_ec",
        "ovr_str004_sec",
        "ovr_str005_ec",
        "ovr_str006_sec",
        "ovr_articulo_8_c",
        "ovr_cs001_sec",
        "ovr_cs002_ec",
        "ovr_cs003_sec",
        'motivo_principal_ovr',
        'motivo_secundario_ovr'

    ]
].copy()

In [8]:
ovr_filtered.rename(
    columns={
        "parent_p_subsidiaria_s":"parent_sub",
        "issuerid":"brs_id",
        "permid_clarity":"permid",
        "parent_name_aladdin" : "parent_name_brs",
        "parent_issuerid_aladdin" : "parent_brs_id",
        "ovr_str001_sec" : "str_001",
        "ovr_str002_ec" : "str_002",
        "ovr_str003_ec" : "str_003",
        "ovr_str003b_ec" : "str_003b",
        "ovr_str004_sec" : "str_004",
        "ovr_str005_ec" : "str_005",
        "ovr_str006_sec" : "str_006",
        "ovr_articulo_8_c" : "art_8", 
        "ovr_cs001_sec" : "cs_001",
        "ovr_cs002_ec" : "cs_003",
        "ovr_cs003_sec" : "cs_002"
        }, 
        inplace=True
)

In [27]:
#get crossreference
crossreference = pd.read_csv(r"C:\Users\n740789\Documents\Projects_local\DataSets\crossreference\Aladdin_Clarity_Issuers_20250301.csv", 
                             dtype={"CLARITY_AI": str})

In [10]:
crossreference.columns = ["brs_id", "brs_issuer_name", "permid", "msci_id", "sustainalytics_id"]

In [11]:
# drop from crossreference columns msci_id and susctainalytics_id
crossreference.drop(columns=["msci_id", "sustainalytics_id"], inplace=True)

In [12]:
crossreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134709 entries, 0 to 134708
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   brs_id           134709 non-null  object
 1   brs_issuer_name  134709 non-null  object
 2   permid           131821 non-null  object
dtypes: object(3)
memory usage: 3.1+ MB


In [33]:
# read latest datafeed at the issuer level
target_columns = [
    "issuer_name",
    "permid",
    "clarityid",
    "company_inheriting",
    "parent_company",
    "str_001_s",
    "str_002_ec",
    "str_003_ec",
    "str_003b_ec",
    "str_004_asec",
    "str_005_ec",
    "cs_001_sec",
    "cs_003_sec",
    "cs_002_ec",
    "str_006_sec",
    "art_8_basicos"
]

datafeed = pd.read_csv(
    r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\ficheros_tratados\2025\20250301_Equities_feed_IssuerLevel_sinOVR.csv",
    usecols=target_columns,
    dtype={"permid": str, "clarityid": str}
)

In [34]:
datafeed.shape[0]

69278

69278, 69264, 69259

In [14]:
datafeed.rename(
    columns={
        "str_001_s"  : "str_001",
        "str_002_ec"  : "str_002",
        "str_003_ec"  : "str_003",
        "str_003b_ec"  : "str_003b",
        "str_004_asec"  : "str_004",
        "str_005_ec"  : "str_005",
        "cs_001_sec"  : "str_006",
        "cs_003_sec"  : "art_8", 
        "cs_002_ec"  : "cs_001",
        "str_006_sec"  : "cs_003",
        "art_8_basicos"  : "cs_002"    
    },
    inplace=True
)

In [ ]:
ovr_filtered.info()

In [ ]:
crossreference.info()

In [15]:
# Create a mapping dictionary from permid to brs_id in the crossreference dataframe
mapping = crossreference.set_index('permid')['brs_id'].to_dict()

# Use the mapping to fill missing brs_id values in ovr_filtered based on the permid column
ovr_filtered['brs_id'] = ovr_filtered['brs_id'].fillna(ovr_filtered['permid'].map(mapping))

In [ ]:
for c in ovr_filtered.columns:
    print(c)

In [16]:
# List of columns to check
columns_to_check = [
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8", 
    "cs_001",
    "cs_003",
    "cs_002"
]

# Filter rows where all specified columns are empty or NaN
ovr_clean = ovr_filtered[~ovr_filtered[columns_to_check].apply(lambda row: row.isna().all() or (row == '').all(), axis=1)]

In [ ]:
ovr_clean.info()

In [17]:
df_target_col =[
    "permid",
    "company_inheriting", 
    "parent_company", 
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8", 
    "cs_001",
    "cs_003",
    "cs_002"
]

In [18]:
ovr_merged = pd.merge(ovr_clean, datafeed[df_target_col], how='left', on='permid', suffixes=('_ovr', '_df'))

In [19]:
ovr_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7190 entries, 0 to 7189
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   clarityid              7190 non-null   object
 1   permid                 7190 non-null   object
 2   brs_id                 2703 non-null   object
 3   issuer_name            7190 non-null   object
 4   parent_sub             7190 non-null   object
 5   parent_name_clarity    7189 non-null   object
 6   parent_clarityid       7190 non-null   object
 7   parent_name_brs        7154 non-null   object
 8   parent_brs_id          7151 non-null   object
 9   str_001_ovr            7120 non-null   object
 10  str_002_ovr            7106 non-null   object
 11  str_003_ovr            7099 non-null   object
 12  str_003b_ovr           7095 non-null   object
 13  str_004_ovr            7122 non-null   object
 14  str_005_ovr            7072 non-null   object
 15  str_006_ovr          

In [ ]:
for c in sorted(ovr_merged.columns):
    print(c)

In [20]:
def active_ovr(row, column_pairs):
    result = {}
    for ovr_col, base_col in column_pairs:
        if pd.isna(row[ovr_col]) or pd.isna(row[base_col]):
            result[f"{ovr_col}_active"] = np.nan
        else:
            result[f"{ovr_col}_active"] = row[ovr_col] != row[base_col]
    return result

In [21]:
# Define column pairs to compare based on the provided columns
column_pairs_to_compare = [
    ("str_001_ovr", "str_001_df"),
    ("str_002_ovr", "str_002_df"),
    ("str_003_ovr", "str_003_df"),
    ("str_003b_ovr", "str_003b_df"),
    ("str_004_ovr", "str_004_df"),
    ("str_005_ovr", "str_005_df"),
    ("str_006_ovr", "str_006_df"),
    ("art_8_ovr", "art_8_df"),
    ("cs_001_ovr", "cs_001_df"),
    ("cs_002_ovr", "cs_002_df"),
    ("cs_003_ovr", "cs_003_df")
]

In [22]:

# Add new columns for each override
for index, row in ovr_merged.iterrows():
    active_overrides = active_ovr(row, column_pairs_to_compare)
    for col, value in active_overrides.items():
        ovr_merged.at[index, col] = value

In [23]:
# Fix override grounds
def fix_spell(ground: str) -> str:
    if ground in ['BIC', 'BiC']:
        return 'SR'
    elif ground == 'Exposición':
        return 'exposure'
    elif ground in ['Inheritance', 'Inheritance de subsidiaria', 'Inheritance por caida de SR']:
        return 'inheritance'
    elif ground == 'Comité':
        return 'committee'
    elif ground in ['Controversia', 'Fallo controversias Clarity', 'OVR por controversia nivel 3/5 en Susta pero en Clarity 1/100']:
        return 'controversy'
    else:
        return ground  # Return the original input if no conditions match

ovr_merged.motivo_principal_ovr = overrides.motivo_principal_ovr.apply(fix_spell)

In [24]:
result_df = ovr_merged.copy()

In [25]:
"""
add these columns to result_df:     
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance"
"""
result_df["ovr_grounds_srating"] = result_df["motivo_principal_ovr"] == "SR"
result_df["ovr_grounds_exposure"] = result_df["motivo_principal_ovr"] == "exposure"
result_df["ovr_grounds_controversy"] = result_df["motivo_principal_ovr"] == "controversy"
result_df["ovr_grounds_committee"] = result_df["motivo_principal_ovr"] == "committee"
result_df["ovr_grounds_inheritance"] = result_df["motivo_principal_ovr"] == "inheritance"

ground_columns = [
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance"]

# for every column in in ground_columns, if value is not nan, then set the value to "x"
for col in ground_columns:
    result_df[col] = result_df[col].apply(lambda x: 'x' if x else np.nan)


In [26]:
result_df["ovr_grounds_srating"].value_counts(dropna=False)

ovr_grounds_srating
NaN    7169
x        21
Name: count, dtype: int64

In [36]:
for c in result_df.columns:
    print(c)

clarityid
permid
brs_id
issuer_name
parent_sub
parent_name_clarity
parent_clarityid
parent_name_brs
parent_brs_id
str_001_ovr
str_002_ovr
str_003_ovr
str_003b_ovr
str_004_ovr
str_005_ovr
str_006_ovr
art_8_ovr
cs_001_ovr
cs_003_ovr
cs_002_ovr
motivo_principal_ovr
motivo_secundario_ovr
company_inheriting
parent_company
str_001_df
str_002_df
str_003_df
str_003b_df
str_004_df
str_005_df
str_006_df
art_8_df
cs_001_df
cs_003_df
cs_002_df
str_001_ovr_active
str_002_ovr_active
str_003_ovr_active
str_003b_ovr_active
str_004_ovr_active
str_005_ovr_active
str_006_ovr_active
art_8_ovr_active
cs_001_ovr_active
cs_002_ovr_active
cs_003_ovr_active
ovr_grounds_srating
ovr_grounds_exposure
ovr_grounds_controversy
ovr_grounds_committee
ovr_grounds_inheritance


In [41]:
def save_result_df_to_excel(result_df, output_file, basic_columns):

    aux_col = [
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance",
    "motivo_principal_ovr",
    "motivo_secundario_ovr",
    "company_inheriting",
    "parent_company",
    ]


    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    strategy_columns = [
        "str_001_df", "str_002_df", "str_003_df", "str_003b_df",
        "str_004_df", "str_005_df", "str_006_df", "art_8_df",
        "cs_001_df", "cs_003_df", "cs_002_df",
    ]

    ovr_columns = [
        "str_001_ovr", "str_002_ovr", "str_003_ovr", "str_003b_ovr", 
        "str_004_ovr", "str_005_ovr", "str_006_ovr", "art_8_ovr", 
        "cs_001_ovr", "cs_003_ovr", "cs_002_ovr", 
    ]

    ovr_active_columns = [
        "str_001_ovr_active", "str_002_ovr_active", "str_003_ovr_active", "str_003b_ovr_active", 
        "str_004_ovr_active", "str_005_ovr_active", "str_006_ovr_active", "art_8_ovr_active", 
        "cs_001_ovr_active", "cs_002_ovr_active", "cs_003_ovr_active",     
    ]

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        result_df.to_excel(writer, sheet_name="Full_Data", index=False)

        for strategy, ovr_col, active_col in zip(strategy_columns, ovr_columns, ovr_active_columns):
            result_df["ovr_target"] = strategy
            relevant_columns = basic_columns + ["ovr_target"] + [
                col for col in [strategy, ovr_col, active_col] if col in result_df.columns
            ] + aux_col

            if relevant_columns:
                result_df[relevant_columns].to_excel(writer, sheet_name=f"{strategy}", index=False)

    print(f"Data has been successfully saved to {output_file}")



In [42]:
DATE = datetime.now().strftime("%Y%m%d")

# define basic columns before saving
basic_columns = [
    "clarityid",
    "permid",
    "brs_id",
    "issuer_name",
    "parent_sub",
    "parent_name_clarity",
    "parent_clarityid",
    "parent_name_brs",
    "parent_brs_id",
]
save_result_df_to_excel(result_df, rf"output\{DATE}_override_analysis_subs.xlsx", basic_columns=basic_columns)

Data has been successfully saved to output\20250312_override_analysis_subs.xlsx


In [ ]:
ovr_merged.head()

In [ ]:
result_df.info()

Display results

In [17]:
# merge map_df with datafeed on permid
map_datafeed = pd.merge(overrides, datafeed, on='permid', how='left')

In [18]:
# merge map_df with crossreference on permid
map_datafeed = pd.merge(map_datafeed, crossreference, on='permid', how='left', suffixes=['_x', '_y'])

In [19]:
def drop_columns_by_prefix(df, prefixes):
    columns_to_drop = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    return df.drop(columns=columns_to_drop)

# Drop columns starting with "original_" or "final_"
prefixes_to_remove = ["original_", "final_"]

In [20]:
#rename column permid_x and issuer_name_x map_datafeed to permid and issuer_name
map_datafeed.rename(columns={
    'clarityid_map':'clarityid',
    'issuer_name_map':'issuer_name',
    'clarityid_x':'clarityid',
    'issuer_name_x':'issuer_name',
}, inplace=True)


In [ ]:
map_datafeed.info()

In [22]:
result_target_columns_map = [
    'permid',
    "clarityid",
    'brs_id',
    'issuer_name',
    'brs_issuer_name',
    "company_inheriting",
    "parent_company",
    "parent_name_aladdin",
    "parent_clarityid",
    "ovr_str001",
    "ovr_str002",
    "ovr_str003",
    "ovr_str003b",
    "ovr_str004",
    "ovr_str005",
    "ovr_str006",
    "ovr_cs001",
    "ovr_cs002",
    "ovr_cs003",
    "ovr_art_8",
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8",
    "cs_001",
    "cs_003",
    "cs_002",
    "parent_clarityid",
]

# filter map_datafeed to only include the columns in result_target_columns
result_map_df = map_datafeed[result_target_columns_map].copy()

In [ ]:
for c in result_map_df.columns:
    print(c)

In [24]:
def active_ovr(row, column_pairs):
    result = {}
    for ovr_col, base_col in column_pairs:
        if pd.isna(row[ovr_col]) or pd.isna(row[base_col]):
            result[f"{ovr_col}_active"] = np.nan
        else:
            result[f"{ovr_col}_active"] = row[ovr_col] != row[base_col]
    return result

In [ ]:
# Define column pairs to compare based on the provided columns
column_pairs_to_compare = [
    ("ovr_str001", "str_001"),
    ("ovr_str002", "str_002"),
    ("ovr_str003", "str_003"),
    ("ovr_str003b", "str_003b"),
    ("ovr_str004", "str_004"),
    ("ovr_str005", "str_005"),
    ("ovr_str006", "str_006"),
    ("ovr_cs001", "cs_001"),
    ("ovr_cs002", "cs_002"),
    ("ovr_cs003", "cs_003"),
    ("ovr_art_8", "art_8"),
]

# Add new columns for each override 
for index, row in result_map_df.iterrows():
    active_overrides = active_ovr(row, column_pairs_to_compare)
    for col, value in active_overrides.items():
        result_map_df.at[index, col] = value


In [ ]:
result_map_df.info()

In [27]:
def save_result_df_to_excel(result_df, output_file, basic_columns):

    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    strategy_columns = [
        "str_001", "str_002", "str_003", "str_003b", 
        "str_004", "str_005","str_006", "art_8",
        "cs_001", "cs_002", "cs_003" 
    ]

    ovr_columns = [
        "ovr_str001", "ovr_str002", "ovr_str003", "ovr_str003b", 
        "ovr_str004", "ovr_str005", "ovr_str006", "ovr_art_8",
        "ovr_cs001", "ovr_cs002", "ovr_cs003"
    ]

    ovr_active_columns = [
        "ovr_str001_active", "ovr_str002_active", "ovr_str003_active", "ovr_str003b_active",
        "ovr_str004_active", "ovr_str005_active", "ovr_str006_active", "ovr_art_8_active",
        "ovr_cs001_active", "ovr_cs002_active", "ovr_cs003_active", 
    ]

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        result_df.to_excel(writer, sheet_name="Full_Data", index=False)

        for strategy, ovr_col, active_col in zip(strategy_columns, ovr_columns, ovr_active_columns):
            result_df["ovr_target"] = strategy
            relevant_columns = basic_columns + ["ovr_target"] + [
                col for col in [strategy, ovr_col, active_col] if col in result_df.columns
            ]

            if relevant_columns:
                result_df[relevant_columns].to_excel(writer, sheet_name=f"{strategy}", index=False)

    print(f"Data has been successfully saved to {output_file}")



In [28]:
DATE = datetime.today().strftime('%Y%m%d')

basic_columns = [
    "permid",
    "clarityid",
    "brs_id",
    "issuer_name",
    "brs_issuer_name",
    "company_inheriting",
    "parent_company",
    "parent_name_aladdin",
    "parent_clarityid"
]
#save_result_df_to_excel(result_map_df, rf"output\{DATE}_override_analysis_mapsheet.xlsx", basic_columns=basic_columns)

In [29]:
final_df = result_map_df[basic_columns].copy()

In [ ]:
final_df.head()

In [31]:
final_df["issuer_name"] = final_df["issuer_name"].str.upper().str.strip()
final_df["parent_company"] = final_df["parent_company"].str.upper().str.strip()

In [ ]:
final_df.info()

In [33]:
# merge final_df with crossreference on issuer_name=brs_issuer_name
final_df = pd.merge(final_df, crossreference, left_on="issuer_name", right_on="brs_issuer_name", how="left", suffixes=('_x', '_y'))

In [ ]:
final_df.info()

In [35]:
def fill_missing_values(df, col1, col2):
    """
    For each row:
      - If col1 is missing, fill it from col2.
      - If col2 is missing, fill it from col1.
      - If both are missing, they remain missing.
      - If both have values, no change.
    """
    missing_values = ["", "-", np.nan, pd.NA, None, "NaN", "nan"]

    # Work on a copy to avoid any chained-assignment warnings
    df = df.copy()

    # Convert both columns to string type, then replace any 'missing_values' with np.nan
    df[col1] = df[col1].astype(str).replace(missing_values, np.nan)
    df[col2] = df[col2].astype(str).replace(missing_values, np.nan)

    # 1) Fill col1 from col2
    df[col1] = df[col1].fillna(df[col2])
    # 2) Fill col2 from col1
    df[col2] = df[col2].fillna(df[col1])

    return df



final_df = fill_missing_values(final_df, "permid_x", "permid_y")
final_df = fill_missing_values(final_df, "brs_id_x", "brs_id_y")
final_df = fill_missing_values(final_df, "brs_issuer_name_x", "brs_issuer_name_y")
final_df = fill_missing_values(final_df, "parent_company", "parent_name_aladdin")


In [ ]:
final_df.head()

In [ ]:
final_df[["permid_x","permid_y","brs_id_x","brs_id_y", "clarityid", "issuer_name","brs_issuer_name_x", "brs_issuer_name_y", "company_inheriting","parent_company", "parent_name_aladdin", "parent_clarityid"]].head()

In [38]:
final_df = final_df[["permid_x","brs_id_x", "clarityid", "issuer_name","brs_issuer_name_x", "company_inheriting","parent_company", "parent_name_aladdin", "parent_clarityid"]].copy()


In [41]:
final_df = final_df.loc[:, ~final_df.columns.duplicated()] # Removes one of the duplicate columns


In [ ]:
final_df.head()

In [ ]:
final_df.columns = ["permid","brs_id", "clarityid", "issuer_name","brs_issuer_name", "company_inheriting","parent_company", "parent_name_aladdin", "parent_clarityid"]
final_df.head()

In [ ]:
final_df.info()

In [45]:
# from df_issuers add "isin" to final_df
final_df = final_df.merge(df_security[["permid", "isin"]], on="permid", how="left")
final_df = final_df.merge(df_security[["clarityid", "isin"]], on="clarityid", how="left")

In [46]:
final_df = fill_missing_values(final_df, "isin_x", "isin_y")

In [47]:
# drop isin_y and rename isin_x
final_df.drop(columns=["isin_y"], inplace=True)
final_df.rename(columns={"isin_x":"isin"}, inplace=True)

In [ ]:
final_df.head()

In [ ]:
final_df.info()

In [54]:
missing_permid = final_df[final_df.permid.isna()]

In [55]:
# safe missing permid
missing_permid.to_excel(rf"output\{DATE}_missing_permid.xlsx", index=False)

In [49]:
# safe final_df to csv
final_df.to_csv(rf"output\{DATE}_final_mapeo.csv", index=False)

In [45]:
# Filtering: One column is empty (NaN or ""), but not both at the same time
#filtered_df = final_df[(final_df["brs_id_x"].isna() | (final_df["brs_id_x"] == "") | (final_df["brs_id_x"]=="-")) &
#                    ~(final_df["brs_id_y"].isna() | (final_df["brs_id_y"] == "") | (final_df["brs_id_y"]=="-"))].copy()


In [46]:
#filtered_df.info()

In [47]:
# remove max col display 
pd.options.display.max_rows = 200
#filtered_df[["permid","permid_crossref","clarityid","brs_id", "brs_id_crossref","issuer_name", "brs_issuer_name","brs_issuer_name_crossref"]].head(100)

In [48]:
# save filtered_df to csv
#filtered_df[["permid","permid_crossref","clarityid","brs_id", "brs_id_crossref","issuer_name", "brs_issuer_name","brs_issuer_name_crossref"]].to_csv(rF"output\{DATE}_gap_issuer_id_aladdin.csv", index=False)